# Create datasets for training and validation

In [1]:
import pandas as pd
import numpy as np
import re
import os

## Read target feature

In [2]:
target_feature = pd.read_csv('../target_feature/01_create_target_feature.csv',
                             sep=';')

In [5]:
target_feature.head()

,folder,file_name,number_of_words,pct_pol_neg,pct_pol_pos,pct_pol_neu
0,ciclo_3,8998-Santo Antônio de Jesus-BA.pdf,45543,0.015063,0.032302,0.954087
1,ciclo_3,9024-Ulianópolis-PA.pdf,17432,0.018945,0.022160,0.959642
2,ciclo_3,9010-Aldeias Altas-MA.pdf,59605,0.022763,0.024140,0.954407
3,ciclo_3,9034-Paraíba do Sul-RJ.pdf,15486,0.014342,0.029007,0.957103
4,ciclo_3,9045-Governador Celso Ramos-SC.pdf,5177,0.011985,0.025130,0.963657


In [6]:
target_feature['temp'] = target_feature['file_name'].str.replace('[0-9]|.pdf|-', ' ', regex=True)\
    .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
target_feature['city'] = target_feature['temp'].str[:-3]
target_feature['state'] = target_feature['temp'].str[-2:]
target_feature['city_state'] = target_feature['city'].map(str) + '_' + target_feature['state']

target_feature = target_feature.drop("temp", axis=1)
target_feature.head()

,folder,file_name,number_of_words,pct_pol_neg,pct_pol_pos,pct_pol_neu,city,state,city_state
0,ciclo_3,8998-Santo Antônio de Jesus-BA.pdf,45543,0.015063,0.032302,0.954087,santo antonio de jesus,ba,santo antonio de jesus_ba
1,ciclo_3,9024-Ulianópolis-PA.pdf,17432,0.018945,0.022160,0.959642,ulianopolis,pa,ulianopolis_pa
2,ciclo_3,9010-Aldeias Altas-MA.pdf,59605,0.022763,0.024140,0.954407,aldeias altas,ma,aldeias altas_ma
3,ciclo_3,9034-Paraíba do Sul-RJ.pdf,15486,0.014342,0.029007,0.957103,paraiba do sul,rj,paraiba do sul_rj
4,ciclo_3,9045-Governador Celso Ramos-SC.pdf,5177,0.011985,0.025130,0.963657,governador celso ramos,sc,governador celso ramos_sc


## Read explanatory features: education, family, fertility and work (2000 and 2010)

In [7]:
raw_dataset = target_feature

In [8]:
state_name_to_acronym = pd.DataFrame({'full_state_name': 
                                      ['acre', 
                                       'alagoas', 
                                       'amapa', 
                                       'amazonas', 
                                       'bahia', 
                                       'ceara', 
                                       'distrito_federal', 
                                       'espirito_santo', 
                                       'goias', 
                                       'maranhao', 
                                       'mato_grosso', 
                                       'mato_grosso_do_sul', 
                                       'minas_gerais', 
                                       'para', 
                                       'paraiba', 
                                       'parana', 
                                       'pernambuco', 
                                       'piaui', 
                                       'rio_de_janeiro', 
                                       'rio_grande_do_norte', 
                                       'rio_grande_do_sul', 
                                       'rondonia', 
                                       'roraima', 
                                       'santa_catarina', 
                                       'sao_paulo', 
                                       'sergipe', 
                                       'tocantins'],
                                      'acronym': ['ac',
                                                  'al',
                                                  'ap',
                                                  'am',
                                                  'ba',
                                                  'ce',
                                                  'df',
                                                  'es',
                                                  'go',
                                                  'ma',
                                                  'mt',
                                                  'ms',
                                                  'mg',
                                                  'pa',
                                                  'pb',
                                                  'pr',
                                                  'pe',
                                                  'pi',
                                                  'rj',
                                                  'rn',
                                                  'rs',
                                                  'ro',
                                                  'rr',
                                                  'sc',
                                                  'sp',
                                                  'se',
                                                  'to']})

var_list = ['var_01',
            'var_02',
            'var_03']

In [9]:
paths = ['../ibge_censo/2000/education',
         '../ibge_censo/2000/family',
         '../ibge_censo/2000/fertility',
         '../ibge_censo/2000/work',
         '../ibge_censo/2010/education',
         '../ibge_censo/2010/family',
         '../ibge_censo/2010/fertility',
         '../ibge_censo/2010/work']

In [10]:
for path in paths:
    
    for var_name in var_list:
        full_temp = pd.DataFrame()
        
        for state in os.listdir(path):
            if not state.startswith('.'):
                state_acronym = state_name_to_acronym.loc[
                    state_name_to_acronym.full_state_name == state]['acronym'].values[0]
        
                for filename in os.listdir(path + '/' + state):
                    if not filename.startswith('.') and filename.endswith(var_name + '.csv'):
        
                        temp = pd.read_csv(path + '/' + state + '/' + filename)
                        temp['city_state'] = temp['city'].map(str) + '_' + state_acronym
                        
                        full_temp = pd.concat([full_temp, temp])
        
        if full_temp.shape[0] != 0:
            full_temp = full_temp.add_prefix(path.split("/")[2] + '_' + path.split("/")[3] + '_' + var_name + '_')
            column_to_join = path.split("/")[2] + '_' + path.split("/")[3] + '_' + var_name + '_city_state'
            
            raw_dataset = pd.merge(raw_dataset,
                                   full_temp.iloc[:,1:],
                                   left_on="city_state",
                                   right_on=column_to_join,
                                   how="left")
            
            raw_dataset = raw_dataset.drop(column_to_join, axis=1)
            
            print(path + ' [' + var_name + '] ')
        

../ibge_censo/2000/education [var_01] 
../ibge_censo/2000/family [var_01] 
../ibge_censo/2000/family [var_02] 
../ibge_censo/2000/fertility [var_01] 
../ibge_censo/2000/fertility [var_02] 
../ibge_censo/2000/fertility [var_03] 
../ibge_censo/2000/work [var_01] 
../ibge_censo/2000/work [var_02] 
../ibge_censo/2010/education [var_01] 
../ibge_censo/2010/family [var_01] 
../ibge_censo/2010/family [var_02] 
../ibge_censo/2010/fertility [var_01] 
../ibge_censo/2010/fertility [var_02] 
../ibge_censo/2010/fertility [var_03] 
../ibge_censo/2010/work [var_01] 
../ibge_censo/2010/work [var_02] 


## Read explanatory feature: social indicator (not in same pattern as others)

In [11]:
paths = ['../ibge_censo/2010/social_indicator']

### Changing city name due to city being known by two different names

In [13]:
raw_dataset.loc[raw_dataset.file_name=='3238-São Valério da Natividade-TO.pdf', 'city_state'] = 'sao valerio_to'

In [14]:
for path in paths:
    
    for var_name in var_list:
        full_temp = pd.DataFrame()
        
        for state in os.listdir(path):
            if not state.startswith('.'):
                state_acronym = state_name_to_acronym.loc[
                    state_name_to_acronym.full_state_name == state]['acronym'].values[0]
                
                for filename in os.listdir(path + '/' + state):
                    if not filename.startswith('.') and filename.endswith(var_name + '.csv'):
                        
                        temp = pd.read_csv(path + '/' + state + '/' + filename)
                        temp['city_state'] = temp['city'].map(str) + '_' + state_acronym
                        
                        full_temp = pd.concat([full_temp, temp])
                        
        if full_temp.shape[0] != 0:
            full_temp = full_temp.add_prefix(path.split("/")[3] + '_' + var_name + '_')
            column_to_join = path.split("/")[3] + '_' + var_name + '_city_state'
            
            raw_dataset = pd.merge(raw_dataset,
                                   full_temp.iloc[:,1:],
                                   left_on="city_state",
                                   right_on=column_to_join,
                                   how="left")
            
            raw_dataset = raw_dataset.drop(column_to_join, axis=1)
            
            print(path + ' [' + var_name + '] ')
        

../ibge_censo/2010/social_indicator [var_01] 
../ibge_censo/2010/social_indicator [var_02] 
../ibge_censo/2010/social_indicator [var_03] 


In [15]:
for c in raw_dataset.columns:
    print(c)

folder
file_name
number_of_words
pct_pol_neg
pct_pol_pos
pct_pol_neu
city
state
city_state
2000_education_var_01_quantity
2000_family_var_01_total
2000_family_var_01_adequada
2000_family_var_01_semi_adequada
2000_family_var_01_inadequada
2000_family_var_02_qt
2000_fertility_var_01_total
2000_fertility_var_01_has_children
2000_fertility_var_01_children_born
2000_fertility_var_01_children_borned_live
2000_fertility_var_01_children_borned_dead
2000_fertility_var_02_total
2000_fertility_var_02_married
2000_fertility_var_02_separated
2000_fertility_var_02_divorced
2000_fertility_var_02_widow
2000_fertility_var_02_single
2000_fertility_var_03_total
2000_work_var_01_total
2000_work_var_01_domestic_regular
2000_work_var_01_domestic_irregular
2000_work_var_01_other_regular
2000_work_var_01_military_and_gov
2000_work_var_01_other_irregular
2000_work_var_02_total
2000_work_var_02_regular
2000_work_var_02_military_and_gov
2000_work_var_02_irregular
2000_work_var_02_employers
2000_work_var_02_entre

In [16]:
raw_dataset.to_csv('02_data_processing_01_raw_dataset.csv',
                   sep=';',
                   index=False)

## Feature engineering

In [18]:
full_dataset = pd.read_csv('02_data_processing_01_raw_dataset.csv',
                           sep=';')

In [19]:
full_dataset = full_dataset.replace('-', 0)

full_dataset.iloc[:, 9:] = full_dataset.iloc[:, 9:].apply(pd.to_numeric)

For all the features created using IBGE, divide them from the position in 2000 by the position in 2010, with the following observations: **if the feature is not a proportion, then divide the feature by the population size of the year (2000 or 2010)**

In [20]:
full_dataset['education_var_01_qt_pct'] = (full_dataset['2000_education_var_01_quantity'] / full_dataset['2000_family_var_02_qt']) / (full_dataset['2010_education_var_01_quantity'] / full_dataset['2010_family_var_02_qt'])

full_dataset['family_var_01_adequada_pct'] = (full_dataset['2000_family_var_01_adequada'] / full_dataset['2000_family_var_01_total']) / (full_dataset['2010_family_var_01_adequada'] / full_dataset['2010_family_var_01_total'])
full_dataset['family_var_01_semi_adequada_pct'] = (full_dataset['2000_family_var_01_semi_adequada'] / full_dataset['2000_family_var_01_total']) / (full_dataset['2010_family_var_01_semi_adequada'] / full_dataset['2010_family_var_01_total'])
full_dataset['family_var_01_inadequada_pct'] = (full_dataset['2000_family_var_01_inadequada'] / full_dataset['2000_family_var_01_total']) / (full_dataset['2010_family_var_01_inadequada'] / full_dataset['2010_family_var_01_total'])

full_dataset['fertility_var_01_has_children_pct'] = (full_dataset['2000_fertility_var_01_has_children'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_has_children'] / full_dataset['2010_fertility_var_01_total'])
full_dataset['fertility_var_01_children_born_pct'] = (full_dataset['2000_fertility_var_01_children_born'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_children_born'] / full_dataset['2010_fertility_var_01_total'])
full_dataset['fertility_var_01_children_borned_live_pct'] = (full_dataset['2000_fertility_var_01_children_borned_live'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_children_borned_live'] / full_dataset['2010_fertility_var_01_total'])
full_dataset['fertility_var_01_children_borned_dead_pct'] = (full_dataset['2000_fertility_var_01_children_borned_dead'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_children_borned_dead'] / full_dataset['2010_fertility_var_01_total'])

full_dataset['fertility_var_02_married_pct'] = (full_dataset['2000_fertility_var_02_married'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_married'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_separated_pct'] = (full_dataset['2000_fertility_var_02_separated'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_separated'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_divorced_pct'] = (full_dataset['2000_fertility_var_02_divorced'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_divorced'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_widow_pct'] = (full_dataset['2000_fertility_var_02_widow'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_widow'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_single_pct'] = (full_dataset['2000_fertility_var_02_single'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_single'] / full_dataset['2010_fertility_var_02_total'])

full_dataset['fertility_var_03_total_pct'] = (full_dataset['2000_fertility_var_03_total'] / full_dataset['2000_family_var_02_qt']) / (full_dataset['2010_fertility_var_03_total'] / full_dataset['2010_family_var_02_qt'])

full_dataset['work_var_01_regular_pct'] = ((full_dataset['2000_work_var_01_domestic_regular'] + full_dataset['2000_work_var_01_other_regular'] + full_dataset['2000_work_var_01_military_and_gov']) / full_dataset['2000_work_var_01_total']) / ((full_dataset['2010_work_var_01_main_regular'] + full_dataset['2010_work_var_01_other_regular']) / full_dataset['2010_work_var_01_total'])
full_dataset['work_var_01_irregular_pct'] = ((full_dataset['2000_work_var_01_domestic_irregular'] + full_dataset['2000_work_var_01_other_irregular']) / full_dataset['2000_work_var_01_total']) / ((full_dataset['2010_work_var_01_main_irregular'] + full_dataset['2010_work_var_01_other_irregular']) / full_dataset['2010_work_var_01_total'])

full_dataset['social_indicator_var_01_15_to_24_years_pct'] = (full_dataset['social_indicator_var_01_2000_15_to_24_years'] / full_dataset['social_indicator_var_01_2010_15_to_24_years'])
full_dataset['social_indicator_var_01_25_to_59_years_pct'] = (full_dataset['social_indicator_var_01_2000_25_to_59_years'] / full_dataset['social_indicator_var_01_2010_25_to_59_years'])
full_dataset['social_indicator_var_01_60_to_more_years_pct'] = (full_dataset['social_indicator_var_01_2000_60_to_more_years'] / full_dataset['social_indicator_var_01_2010_60_to_more_years'])

full_dataset['social_indicator_var_02_suitable_pct'] = full_dataset['social_indicator_var_02_2000_suitable'] / full_dataset['social_indicator_var_02_2010_suitable']
full_dataset['social_indicator_var_02_semi_suitable_pct'] = full_dataset['social_indicator_var_02_2000_semi_suitable'] / full_dataset['social_indicator_var_02_2010_semi_suitable']
full_dataset['social_indicator_var_02_inappropriate_pct'] = full_dataset['social_indicator_var_02_2000_inappropriate'] / full_dataset['social_indicator_var_02_2010_inappropriate']

full_dataset['social_indicator_var_03_responsable_illiterate_pct'] = full_dataset['social_indicator_var_03_2000_responsable_illiterate'] / full_dataset['social_indicator_var_03_2010_responsable_illiterate']
full_dataset['social_indicator_var_03_inappropriate_residence_pct'] = full_dataset['social_indicator_var_03_2000_inappropriate_residence'] / full_dataset['social_indicator_var_03_2010_inappropriate_residence']
full_dataset['social_indicator_var_03_responsable_illiterate_and_inappropriate_residence_pct'] = full_dataset['social_indicator_var_03_2000_responsable_illiterate_and_inappropriate_residence'] / full_dataset['social_indicator_var_03_2010_responsable_illiterate_and_inappropriate_residence']

Select only generated features and remove rows with NaN values on it.

In [21]:
modeling_dataset = full_dataset.iloc[:, np.r_[3:6, 7, 88:113]].dropna()

for c in modeling_dataset.columns:
    print(c)

pct_pol_neg
pct_pol_pos
pct_pol_neu
state
education_var_01_qt_pct
family_var_01_adequada_pct
family_var_01_semi_adequada_pct
family_var_01_inadequada_pct
fertility_var_01_has_children_pct
fertility_var_01_children_born_pct
fertility_var_01_children_borned_live_pct
fertility_var_01_children_borned_dead_pct
fertility_var_02_married_pct
fertility_var_02_separated_pct
fertility_var_02_divorced_pct
fertility_var_02_widow_pct
fertility_var_02_single_pct
fertility_var_03_total_pct
work_var_01_regular_pct
work_var_01_irregular_pct
social_indicator_var_01_15_to_24_years_pct
social_indicator_var_01_25_to_59_years_pct
social_indicator_var_01_60_to_more_years_pct
social_indicator_var_02_suitable_pct
social_indicator_var_02_semi_suitable_pct
social_indicator_var_02_inappropriate_pct
social_indicator_var_03_responsable_illiterate_pct
social_indicator_var_03_inappropriate_residence_pct
social_indicator_var_03_responsable_illiterate_and_inappropriate_residence_pct


In [22]:
modeling_dataset.head()

,pct_pol_neg,pct_pol_pos,pct_pol_neu,state,education_var_01_qt_pct,family_var_01_adequada_pct,family_var_01_semi_adequada_pct,family_var_01_inadequada_pct,fertility_var_01_has_children_pct,fertility_var_01_children_born_pct,...,work_var_01_irregular_pct,social_indicator_var_01_15_to_24_years_pct,social_indicator_var_01_25_to_59_years_pct,social_indicator_var_01_60_to_more_years_pct,social_indicator_var_02_suitable_pct,social_indicator_var_02_semi_suitable_pct,social_indicator_var_02_inappropriate_pct,social_indicator_var_03_responsable_illiterate_pct,social_indicator_var_03_inappropriate_residence_pct,social_indicator_var_03_responsable_illiterate_and_inappropriate_residence_pct
0,0.015063,0.032302,0.954087,ba,1.084266,0.702436,1.331139,1.884205,0.947679,1.135953,...,0.493153,2.126923,1.495353,1.239078,0.773467,1.470543,1.173837,1.774015,1.306122,1.859002
1,0.018945,0.022160,0.959642,pa,0.845772,0.000000,0.832523,2.168407,1.246863,1.520075,...,0.711015,3.229858,1.807236,1.121196,0.000000,0.251064,3.458490,2.107538,3.520115,6.327869
2,0.022763,0.024140,0.954407,ma,1.105667,0.240849,0.800463,1.807281,0.950325,1.079955,...,0.420519,2.049849,1.341394,1.058180,0.445783,0.567587,1.392267,1.316604,1.325667,1.626568
3,0.014342,0.029007,0.957103,rj,0.887970,0.807424,1.340452,2.270949,1.010823,1.254698,...,0.619561,3.018519,2.041667,1.646512,0.891188,1.282571,1.817568,2.159142,1.831304,2.352113
4,0.011985,0.025130,0.963657,sc,1.052355,0.775598,1.428189,inf,1.019168,1.130896,...,0.356448,1.431655,1.621239,1.205721,0.851144,1.320743,2.117647,1.674837,3.416667,inf


In [23]:
modeling_dataset.to_csv('02_data_processing_02_modeling_dataset.csv',
                        sep=';',
                        index=False)

## Splitting the datasets (training and validation)

In [33]:
training_dataset = modeling_dataset.sample(frac=0.75,
                                           random_state=7)
validation_dataset = modeling_dataset.drop(training_dataset.index)

In [34]:
print('=== Number of rows === \n' +
      'Training: ' + str(len(training_dataset)) + '\n' +
      'Validation: ' + str(len(validation_dataset)))

=== Number of rows === 
Training: 382
Validation: 127


In [35]:
training_dataset.to_csv('02_data_processing_03_training_dataset.csv',
                        sep=';',
                        index=False)

validation_dataset.to_csv('02_data_processing_04_validation_dataset.csv',
                          sep=';',
                          index=False)